# Классификация: SI > медианы

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Подготовка целевой переменной
y = (df["SI"] > df['SI'].median()).astype(int)
X = df[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение моделей
logit = LogisticRegression(max_iter=1000)
logit.fit(X_train_scaled, y_train)
y_pred_logit = logit.predict(X_test_scaled)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
y_pred_lgb = lgb.predict(X_test)

# Оценка
for name, preds, clf, xtest in [("Logistic Regression", y_pred_logit, logit, X_test_scaled),
                                ("Random Forest", y_pred_rf, rf, X_test),
                                ("XGBoost", y_pred_xgb, xgb, X_test),
                                ("LightGBM", y_pred_lgb, lgb, X_test)]:
    print(f"{name}: Accuracy={accuracy_score(y_test, preds):.3f}, Precision={precision_score(y_test, preds):.3f}, Recall={recall_score(y_test, preds):.3f}, F1={f1_score(y_test, preds):.3f}, AUC={roc_auc_score(y_test, clf.predict_proba(xtest)[:, 1]):.3f}")


C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [12:00:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 399, number of negative: 399
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002206 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18096
[LightGBM] [Info] Number of data points in the train set: 798, number of used features: 168
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

# Классификация: предсказание среднего уровня избирательности (SI > медианы)

В этом разделе выполняются следующие шаги:

1. **Формирование целевой переменной**  
   - Рассчитываем медиану SI (`df["SI"].median()`).  
   - Создаём бинарный вектор `y`, где  
     - `1` = «SI > медианы» (более селективные соединения),  
     - `0` = «SI ≤ медианы» (менее селективные).

2. **Разделение выборки и масштабирование**  
   - Стратифицированный сплит на тренировочную и тестовую выборки  
     (`train_test_split(..., stratify=y, test_size=0.2)`), чтобы сохранить соотношение классов.  
   - Масштабируем признаки через `StandardScaler` для моделей, чувствительных к шкале.

3. **Обучение и оценка моделей**  
   - **LogisticRegression**, **RandomForestClassifier**, **XGBClassifier**, **LGBMClassifier**.  
   - Метрики на тестовой выборке:

   | Модель               | Accuracy | Precision | Recall | F1    | ROC AUC |
   |----------------------|----------|-----------|--------|-------|---------|
   | Logistic Regression  | 0.620    | 0.622     | 0.610  | 0.616 | 0.665   |
   | Random Forest        | 0.630    | 0.635     | 0.610  | 0.622 | 0.675   |
   | XGBoost              | 0.610    | 0.615     | 0.590  | 0.602 | 0.649   |
   | LightGBM             | 0.635    | 0.642     | 0.610  | 0.626 | 0.665   |

4. **Анализ результатов**  
   - Все модели демонстрируют скромный рост над базовым 50% (случайным угадыванием).  
   - **LightGBM** лидирует по Accuracy (0.635) и F1-score (0.626), а также по Precision (0.642).  
   - **Random Forest** показывает наилучшее ROC AUC (0.675), то есть лучшую способность ранжировать образцы по вероятности высокого SI.  
   - **XGBoost** чуть отстаёт по всем метрикам, но остаётся конкурентоспособным.

> **Вывод:** задача классификации «SI > медианы» оказалась достаточно сложной — модели достигают Accuracy ~0.62–0.64 и ROC AUC ~0.65–0.68. Среди них **LightGBM** обеспечивает лучший баланс точности и полноты, а **Random Forest** — лучшее ранжирование по вероятности. Для повышения качества стоит рассмотреть расширенный подбор гиперпараметров и дополнительные признаки.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
df = pd.read_excel("Данные_для_курсовои_Классическое_МО.xlsx")
df.drop(columns=["Unnamed: 0"], inplace=True)
constant_cols = [col for col in df.columns if df[col].nunique() == 1]
df.drop(columns=constant_cols, inplace=True)
df.dropna(inplace=True)
features = [col for col in df.columns if col not in ["IC50, mM", "CC50, mM", "SI"]]
scaler = StandardScaler()
